In [ ]:
import numpy as np
import os
import pickle
import cv2
import numpy as np
import argparse
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 
from sklearn.feature_selection import SelectKBest, f_classif, chi2
from sklearn.metrics import *
import random
import pandas as pd
from tqdm import tqdm 
import re
from scipy.ndimage import gaussian_filter1d
from scipy.ndimage import gaussian_filter
import scipy.signal

In [ ]:
!pip install scikit-learn==0.23.2

## GENERATE PAIR INDEXES FOR LFW

In [ ]:
# pairindexes contains a list containing a bunch of [index1, index2]
def genPairIndexes_lfw(attrPath = '/content/drive/MyDrive/LFW/metadata/modified_attributes.txt', genLabels = 0):
    global df_attributes
    df_attributes = pd.read_csv(attrPath, delimiter = ",")
    output_low_level_path = '/content/drive/MyDrive/project-sepnu/data/low level/lfw/'
    file_images_under_error = open(output_low_level_path + "images_under_error.txt", "r")
    while True:
        line = file_images_under_error.readline().replace("\n", "")
        if line == "": break
        line = line.split()[0]
        matched_pattern = re.match(r"^([\w-]+?)_(\d+).jpg$", line[line.rindex("/")+1:])
        name, image_num = matched_pattern.groups()
        name = name.replace("_", " ")
        image_num = int(image_num)
        initial_shape = df_attributes.shape
        df_attributes.drop(df_attributes[(df_attributes['person'] == name) & (df_attributes['imagenum'] == image_num)].index, inplace = True)
        final_shape = df_attributes.shape
        assert (final_shape[0] + 1, final_shape[1]) == initial_shape
    file_images_under_error.close()
    assert df_attributes.shape[0] == np.load(output_low_level_path + "rgb_chin.npy").shape[0]

    df = df_attributes
    lfwnames = df['person'].tolist()
    lfwnames_nums = df['imagenum'].tolist()
    lfwnames_act = []
    for i in range(len(lfwnames)):
        num = '{0:04}'.format(lfwnames_nums[i])
        lfwnames_act.append(lfwnames[i].replace(" ","_") + "_" + num + ".jpg")
    file_images_under_error = open(output_low_level_path + "images_under_error.txt", "r")

    with open("/content/drive/MyDrive/project-sepnu/src/verification /dataset.pkl", "rb") as input_file:
        datalist = pickle.load(input_file)

    pairIndexes = []
    labels = [] 
    err_names = {'dummy.jpg'}
    for first_person,second_person,label in datalist:
        first_person = first_person.split("/")[2]
        second_person = second_person.split("/")[2]
        if first_person not in lfwnames_act :
            # print(first_person, "not in list")
            err_names.add(first_person)
        elif second_person not in lfwnames_act:
            # continue
            # print(second_person, "not in list")
            err_names.add(second_person)
        else:
            # print(lfwnames_act.index(first_person), lfwnames_act.index(second_person))
            pairIndexes.append([lfwnames_act.index(first_person), lfwnames_act.index(second_person)])
            if label == 0:
                labels.append(-1)
            else:
                labels.append(1)
    # for i in range(10):
    # print(err_names)
        # print(pairIndexes[i])
    if genLabels:
        return pairIndexes, err_names, labels
    else:
        return pairIndexes, err_names

## ATTRIBUTE TRAITS GENERATOR

In [ ]:
from tqdm import tqdm_notebook
def run_attribute_SVMs_lfw(pair_indexes):
    global df_attributes
    output_low_level_path = '/content/drive/MyDrive/project-sepnu/data/low level/lfw/'
    
    ## this is for sttribute classifiers
    output_file_path = "/content/drive/MyDrive/project-sepnu/data/attribute classifier/lfw/"
    map_attributes_features_path = output_file_path + "map_attributes_features.txt"
    file_map_attributes_features = open(map_attributes_features_path, "r")
    map_attributes_features = {}
    for attribute, features in [line.split(":") for line in file_map_attributes_features.readlines()]:
        map_attributes_features[attribute] = features.split()
    file_map_attributes_features.close()

    final_preds = []

    for person1,person2 in tqdm_notebook(pair_indexes):
        person1preds,person2preds = [],[]
        for attribute, features in map_attributes_features.items():
            if len(features) == 0: continue
            final_feature_set = []
            p1feature = np.array([])
            p2feature = np.array([])
            for feature in features:
                feature_array = np.load(output_low_level_path + feature + ".npy")
                p1feature = np.hstack((p1feature,feature_array[person1]))
                p2feature = np.hstack((p2feature,feature_array[person2]))
            final_feature_set.extend([np.array(p1feature),np.array(p2feature)])
            final_feature_set = np.array(final_feature_set)
            with open(output_file_path + attribute + "_svm_model.pkl", "rb") as f: 
                svm_obj = pickle.load(f)
            feature_preds = svm_obj.decision_function(final_feature_set)
            person1preds.append(feature_preds[0])
            person2preds.append(feature_preds[1])
        final_preds.append([person1preds,person2preds])

    return final_preds

## SIMILE TRAITS GENERATOR

In [ ]:
output_file_path, output_low_level_path, list_ref_persons, ref_file_path  = None, None, None, None
dict_facepart_features = {"eyes":["rgb_left_eye", "hsv_left_eye", "grad_mag_left_eye", "grad_orien_left_eye", "rgb_right_eye", "hsv_right_eye", "grad_mag_right_eye", "grad_orien_right_eye"], "nose":["rgb_nose", "hsv_nose", "grad_mag_nose", "grad_orien_nose"], "mouth":["rgb_mouth", "hsv_mouth", "grad_mag_mouth", "grad_orien_mouth"]}

def run_simile_SVMs_lfw(pair_indexes):
    output_file_path = "/content/drive/MyDrive/project-sepnu/data/simile classifier/celeb_face_recog/"
    output_low_level_path = "/content/drive/MyDrive/project-sepnu/data/low level/lfw/"
    ref_file_path = "/content/drive/MyDrive/CelebrityFaceRecognition/reference_faces/"
    file_ref_persons = open(ref_file_path + "reference_faces.txt", "r")
    list_ref_persons = [ref_person.replace("\n", "") for ref_person in file_ref_persons.readlines()]
    file_ref_persons.close()
    final_preds = []
    # global output_file_path, output_low_level_path, list_ref_persons, ref_file_path
    cnt = 0
    for person1,person2 in tqdm_notebook(pair_indexes):
        person1preds = []
        person2preds = []
        for ref_person in list_ref_persons:
            for attribute, features in dict_facepart_features.items():
                final_feature_set = []
                p1feature = []
                p2feature = []
                for feature in features:
                    feature_array = np.load(output_low_level_path + feature + ".npy")
                    p1feature.extend(feature_array[person1]/feature_array[person1].sum())
                    p2feature.extend(feature_array[person2]/feature_array[person2].sum())
                
                p1feature = np.array(p1feature)
                p2feature = np.array(p2feature)

                p1feature = p1feature[::2] + p1feature[1::2]
                p2feature = p2feature[::2] + p2feature[1::2]
                final_feature_set.extend([p1feature,p2feature])
                final_feature_set = np.array(final_feature_set)
                with open(output_file_path + ref_person + "_" + attribute + "_svm_model.pkl", "rb") as f: 
                    svm_obj = pickle.load(f)
                feature_preds = svm_obj.predict(final_feature_set)
                person1preds.append(feature_preds[0])
                person2preds.append(feature_preds[1])
        
        final_preds.append([person1preds,person2preds])
        cnt += 1
        if cnt == 1000:
            cnt = 0
            with open("/content/drive/MyDrive/project-sepnu/data/verification/simile_lfw_out.pkl","wb") as f:
                pickle.dump(final_preds,f)
            print("1000 done")
    return final_preds


In [ ]:
pairIndexes,err_names = genPairIndexes_lfw()
out = run_simile_SVMs_lfw(pairIndexes)
with open("/content/drive/MyDrive/project-sepnu/data/verification/simile_lfw_out.pkl") as f:
    pickle.dump(out,f)

## FINAL VERIFICATION CLASSIFIER CODE

In [ ]:
# add both PKL files together and save them

with open("/content/drive/MyDrive/project-sepnu/data/verification/simile_lfw_out3.pkl", 'rb') as file:
    simileFeats = pickle.load(file)
with open("/content/drive/MyDrive/project-sepnu/data/verification/attr_lfw_out1.pkl", 'rb') as file:
    attrFeats = pickle.load(file)

simileFeats = np.array(simileFeats)
attrFeats = np.array(attrFeats)
print(simileFeats.shape)
print(attrFeats.shape)
totalFeats = np.concatenate((simileFeats, attrFeats),axis=2)
print(totalFeats.shape)

with open("/content/drive/MyDrive/project-sepnu/data/verification/total_lfw_out1.pkl", 'wb') as file:
    pickle.dump(totalFeats, file)

(6143, 2, 36)
(6143, 2, 20)
(6143, 2, 56)


In [ ]:
def calcGaussian(people1, people2):
    subtr = people1 - people2
    mult = np.multiply(people1, people2)
    avg = people1 - people2
    r, c = subtr.shape
    subtrG = gaussian_filter1d(subtr, sigma = 1)
    subtrG1 = gaussian_filter(subtr, sigma = 1)
    subtrG2 = gaussian_filter(np.ravel(subtr), sigma = 1).reshape(r, c)
    window = scipy.signal.gaussian(r*c, 1)
    subtrG3 = np.multiply(np.array(window), np.ravel(subtr)).reshape(r,c)

    x = np.exp(-((subtr + mult)**2)/(2))
    subtrG4 = subtr * x
    subtrG5 = subtr * (x/np.linalg.norm(x))
    multG = mult * x
    
    # pairs = np.empty((people1.shape[0], people1.shape[1]))
    pairs = []

    # pairs.append(np.concatenate([subtr, mult, avg],axis=1))
    pairs.append(np.concatenate([subtrG4, multG],axis=1))
    pairs = np.array(pairs)
    r, c, d = pairs.shape
    pairs.resize(c,d)

    return pairs

In [ ]:
def applyGeometricTransform(onlyTen=0, totalFeatsPath = "/content/drive/MyDrive/project-sepnu/data/verification/total_lfw_out.pkl"):
    with open(totalFeatsPath, 'rb') as file:
        totalFeats = pickle.load(file)
    totalFeats = np.array(totalFeats)
    if onlyTen:
        totalFeats = totalFeats[:100,:,:]
    # print(attrFeats.shape)
    people1 = totalFeats[:,0,:]
    people2 = totalFeats[:,1,:]
    pairs = calcGaussian(people1, people2)
    return pairs, totalFeats

In [ ]:
def learn_SVM(outputPath = '/content/drive/MyDrive/project-sepnu/data/verification'):
    pairs, totalFeats = applyGeometricTransform()
    _, _, labels = genPairIndexes_lfw(genLabels=1)
    # labels = np.random.choice(labels, 100, replace=False)
    print('labels size: ',np.array(labels).shape)
    print('trait vect size: ', np.array(pairs).shape)   
    # print(totalFeats[0][1])
    X_train, X_test, y_train, y_test = train_test_split(pairs, labels, test_size = 0.3)
    np.save(os.path.join(outputPath, "_X_train"), X_train)
    np.save(os.path.join(outputPath, "_y_train"), y_train)
    np.save(os.path.join(outputPath, "_X_test"), X_test)
    np.save(os.path.join(outputPath, "_y_test"), y_test)
    svm_obj = svm.SVC()
    grid_values = {'C':[0.01, 0.1, 1, 5, 10, 15, 20, 25, 30, 50]}
    grid_clf_acc = GridSearchCV(svm_obj, param_grid = grid_values, scoring = 'accuracy')
    grid_clf_acc.fit(X_train, y_train)
    with open(os.path.join(outputPath,'verification_classifier.pkl'), "wb") as f: 
        pickle.dump(grid_clf_acc, f) 
       

In [ ]:
learn_SVM()

labels size:  (6143,)
trait vect size:  (6143, 112)


KeyboardInterrupt: ignored

In [ ]:
def test_SVM(outputPath = '/content/drive/MyDrive/project-sepnu/data/verification'):
    with open(os.path.join(outputPath,"verification_classifier.pkl"), "rb") as f: 
        clf = pickle.load(f)

    X_test = np.load(os.path.join(outputPath,"_X_test.npy"))
    y_test = np.load(os.path.join(outputPath,"_y_test.npy"))
    predicted_output = clf.predict(X_test)
    print(classification_report(y_test, predicted_output))

In [ ]:
test_SVM()
#OP WITH METHOD2

              precision    recall  f1-score   support

          -1       0.71      0.69      0.72       853
           1       0.59      0.62      0.61       990

    accuracy                           0.66      1843
   macro avg       0.63      0.62      0.63      1843
weighted avg       0.65      0.66      0.65      1843



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test_SVM()
# OP WITH METHOD1

              precision    recall  f1-score   support

          -1       0.72      0.69      0.71       887
           1       0.61      0.63      0.62       956

    accuracy                           0.67      1843
   macro avg       0.63      0.61      0.62      1843
weighted avg       0.66      0.67      0.64      1843



#### RANDOM

In [ ]:
with open("/content/drive/MyDrive/project-sepnu/data/attribute classifier/lfw/Arched Eyebrows_svm_model.pkl", "rb") as f: 
    svm_obj = pickle.load(f)
te = np.random.rand(2,400)
tem = svm_obj.predict(te)

In [ ]:
lfw = os.listdir("/content/lfw")
lfw[0]
count = 0
for root, dirs, files in os.walk("/content/lfw", topdown=False):
    if(count>10):
        break
    for image in files:
        asdf = os.path.join(root,image)
        print(asdf.replace('jpg','npy'))
    count = count+ 1

In [ ]:
attributeSVM = "/content/drive/MyDrive/project-sepnu/data/attribute classifier/celeba"
simileSVM = "/content/drive/MyDrive/project-sepnu/data/simile classifier/celeb_face_recog"
lfw = os.listdir("/content/lfw")

for root, dirs, images in os.walk("/content/lfw", topdown=False):
    for imagePath in images:
        image=cv2.imread(imagePath)
        trait_class = []
        
        for i, fname in enumerate(os.listdir(attributeSVM)):
            if fname.endswith(".pkl"):
                with open(os.path.join(attributeSVM,fname), 'rb') as file:
                    clf = pickle.load(file)
                svm_op = clf.predict(image)
                trait_class.append(svm_op)
                 
        for i,fname in enumerate(os.listdir(simileSVM)):
            if fname.endswith(".pkl"):
                with open(os.path.join(simileSVM,fname), 'rb') as file:
                    clf = pickle.load(file)
                svm_op = clf.predict(image)
                trait_class.append(svm_op)

        trait_class = np.asarray(trait_class)
        filePath = os.path.join(root, imagePath)
        np.save(filepath.replace('jpg','npy'), trait_class)

In [ ]:
print(X_train.shape)

(2, 4)


In [ ]:
asdf = np.load("/content/AJ_Cook_0001.npy")
print(asdf.shape)

(201,)


In [ ]:

test = np.load("/content/drive/MyDrive/project-sepnu/data/attribute classifier/lfw/Arched Eyebrows_y_train.npy") 
print(test)

[-1  1 -1 ... -1 -1 -1]
